In [39]:
# 모듈 로딩
from sklearn.datasets import fetch_openml
import pandas as pd
import numpy as np

import torch
import torch.nn as nn           # LinearRegression 기능의 클래스 linear
import torch.nn.functional as F # 손실함수
import torch.optim as optim     # 경사하강법 기반의 최적화 관련 모듈
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import TensorDataset # TensorDataset 클래스 로딩

In [40]:
# 데이터 준비
fashion_db = 'fashion-mnist'

fashion_data = fetch_openml(name=fashion_db, parser='auto')

In [41]:
type(fashion_data)

sklearn.utils._bunch.Bunch

In [42]:
#fashion_data.keys()

In [43]:
class DLDataset(Dataset) : #Dataset도... 모듈에 속한 거라서 import 필요했다...;;
    # 초기화 함수
    def __init__(self, x_data, y_data) :
        super().__init__() # 부모클래스의 메서드를 호출 => DLDataset 클래스의 객체를 초기화.
        # x, y 데이터 --> ndarray
        x_data = x_data.values if isinstance(x_data, pd.DataFrame) else x_data
        y_data = y_data.values if isinstance(y_data, pd.DataFrame) else y_data
        
        # x, y 데이터 --> tensor
        self.feature = torch.tensor(x_data)
        self.target = torch.tensor(y_data)
        print('[target & target SHAPE]', self.target.shape, self.target.ndim)

    # 데이터셋의 개수 확인 함수 콜백 함수
    def __len__(self) :
        return self.target.shape[0]
        # 특정 인덱스 데이터+라벨 반환 콜백 함수 (Call back function)
    
    def __getitem__(self, index):
        return self.feature[index], self.target[index]        

In [44]:
# 피처 + 라벨로 분리
feature = fashion_data['data']/255
target = fashion_data['target'].astype(int)

# 피처와 타겟 개수 확인
print(f'FEATURE => {feature.shape}, {feature.ndim}D')
print(f'TARGET => {target.shape}, {target.ndim}D, {target.dtype}')

FEATURE => (70000, 784), 2D
TARGET => (70000,), 1D, int32


In [45]:
# 분류 확인 
print(f'FEATURE => {fashion_data["feature_names"]}')
print(f'TARGET => {fashion_data["target_names"]}')
print(f'CATEGORIES => {fashion_data["categories"]}') # 왜...None..?
print(f'CATEGORIES => {target.unique()}') 
print(f'CATEGORIES => {target.dtypes}') 

FEATURE => ['pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5', 'pixel6', 'pixel7', 'pixel8', 'pixel9', 'pixel10', 'pixel11', 'pixel12', 'pixel13', 'pixel14', 'pixel15', 'pixel16', 'pixel17', 'pixel18', 'pixel19', 'pixel20', 'pixel21', 'pixel22', 'pixel23', 'pixel24', 'pixel25', 'pixel26', 'pixel27', 'pixel28', 'pixel29', 'pixel30', 'pixel31', 'pixel32', 'pixel33', 'pixel34', 'pixel35', 'pixel36', 'pixel37', 'pixel38', 'pixel39', 'pixel40', 'pixel41', 'pixel42', 'pixel43', 'pixel44', 'pixel45', 'pixel46', 'pixel47', 'pixel48', 'pixel49', 'pixel50', 'pixel51', 'pixel52', 'pixel53', 'pixel54', 'pixel55', 'pixel56', 'pixel57', 'pixel58', 'pixel59', 'pixel60', 'pixel61', 'pixel62', 'pixel63', 'pixel64', 'pixel65', 'pixel66', 'pixel67', 'pixel68', 'pixel69', 'pixel70', 'pixel71', 'pixel72', 'pixel73', 'pixel74', 'pixel75', 'pixel76', 'pixel77', 'pixel78', 'pixel79', 'pixel80', 'pixel81', 'pixel82', 'pixel83', 'pixel84', 'pixel85', 'pixel86', 'pixel87', 'pixel88', 'pixel89', 'pixel90', 'pixel9

In [46]:
# 전처리

from sklearn.preprocessing import LabelEncoder

lbel = LabelEncoder()
lbel.fit(target)
targetNP = lbel.transform(target)

print(target.shape, type(target), targetNP.ndim)

(70000,) <class 'pandas.core.series.Series'> 1


In [47]:
my_dataset = DLDataset(feature, targetNP)

[target & target SHAPE] torch.Size([70000]) 1


In [48]:
# 학습용, 검증용, 테스트용 Dataset 

# 파이토치
from torch.utils.data import random_split

seed = torch.Generator().manual_seed(13)
trainDS, validDS, testDS = random_split(my_dataset, [0.7, 0.1, 0.2], generator = seed)
# trainDS 70% + validDS 10% + testDS 20%

print(f'trainDS => {len(trainDS)}개\t validDS => {len(validDS)}개\t testDS => {len(testDS)}개\n')

print(f'[Subset 속성]\nindices : {trainDS.indices}\ndataset : {trainDS.dataset}\n')
print(f'[Subset 속성]\nindices : {validDS.indices}\ndataset : {validDS.dataset}')

trainDS => 49000개	 validDS => 7000개	 testDS => 14000개

[Subset 속성]
indices : [16418, 68417, 10906, 19181, 61630, 25401, 16802, 25287, 27428, 12457, 7937, 49023, 15972, 34342, 17310, 66661, 18374, 56764, 2319, 16933, 23733, 64208, 43704, 46518, 25424, 32414, 66481, 34996, 16687, 16386, 6685, 37913, 47409, 48924, 55402, 55939, 58339, 55941, 7846, 13308, 39150, 56661, 40984, 61133, 8303, 66870, 34427, 33446, 59235, 65966, 15810, 24603, 65957, 10460, 47615, 29788, 13789, 14832, 33875, 17393, 1286, 32707, 62262, 44285, 54367, 53756, 8781, 37890, 47642, 53378, 37999, 39952, 51775, 35952, 58044, 15931, 50893, 35177, 34473, 9900, 1556, 21600, 18597, 33298, 69564, 22863, 46785, 24861, 181, 56035, 54186, 26779, 18218, 27452, 25966, 61031, 47288, 24268, 2489, 9378, 8665, 29034, 35471, 54420, 67385, 1551, 25164, 33488, 68067, 2240, 54826, 5201, 69609, 64265, 3744, 12943, 11714, 226, 59307, 42585, 51906, 22583, 49421, 42421, 10732, 63923, 63120, 66170, 6139, 64273, 1206, 66450, 64586, 10157, 16853,

In [49]:
# DataLoader 생성 : 학습용, 검증용, 테스트용

batch = 100
trainDL =DataLoader(trainDS, batch_size=batch)
validDL =DataLoader(validDS, batch_size=batch)
testDL =DataLoader(testDS, batch_size=batch)

len(trainDL), len(validDL), len(testDL)

(490, 70, 140)

In [50]:
#print(f'batch_size : {batch}\n')
#print(f'trainDS => {len(trainDS)}개\t validDS => {len(validDS)}개\t testDS => {len(testDS)}개\n')
#print(f'trainDL => {len(trainDL)}개\t validDS => {len(validDL)}개\t testDS => {len(testDL)}개\n')

In [51]:
# DataLoader 속성

for _, (feature, target) in enumerate(trainDL) :
    print(f'[{_+1}] feature {feature.shape}')

[1] feature torch.Size([100, 784])


[2] feature torch.Size([100, 784])
[3] feature torch.Size([100, 784])
[4] feature torch.Size([100, 784])
[5] feature torch.Size([100, 784])
[6] feature torch.Size([100, 784])
[7] feature torch.Size([100, 784])
[8] feature torch.Size([100, 784])
[9] feature torch.Size([100, 784])
[10] feature torch.Size([100, 784])
[11] feature torch.Size([100, 784])
[12] feature torch.Size([100, 784])
[13] feature torch.Size([100, 784])
[14] feature torch.Size([100, 784])
[15] feature torch.Size([100, 784])
[16] feature torch.Size([100, 784])
[17] feature torch.Size([100, 784])
[18] feature torch.Size([100, 784])
[19] feature torch.Size([100, 784])
[20] feature torch.Size([100, 784])
[21] feature torch.Size([100, 784])
[22] feature torch.Size([100, 784])
[23] feature torch.Size([100, 784])
[24] feature torch.Size([100, 784])
[25] feature torch.Size([100, 784])
[26] feature torch.Size([100, 784])
[27] feature torch.Size([100, 784])
[28] feature torch.Size([100, 784])
[29] feature torch.Size([100, 784])


In [52]:
# 모델 클래스 정의
# 클래스명 : CMdl 
class CMdl(nn.Module) :
    
    # 구성 요소 정의
    def __init__(self, in_, out_) :
        super().__init__()
        self.input_layer = nn.Linear(in_, 200)
        self.relu=nn.ReLU() 
        self.hidden_layer=nn.Linear(200, 100)
        self.output_layer = nn.Linear(100, out_)
    
    # 순방향 학습 진행 함수
    def forward(self, x):
        y = self.input_layer(x) #
        y = self.ReLU(y)            # 
        y = self.hidden_layer(y)    # 
        y = self.ReLU(y)            # 
        y = self.output_layer(y)    # 

        return y

In [53]:
# 실생 디바이스 실행.
DEVICE = 'cuda'if torch.cuda.is_available() else 'cpu'

# 학습 횟수
EPOCHS = 20

In [54]:
IN, OUT = my_dataset.feature.shape[1], len(np.unique(targetNP))
print(IN, OUT)
model = CMdl(IN, OUT).to(DEVICE)

784 10


In [55]:
# 손실 함수
LOSS_FN = nn.CrossEntropyLoss().to(DEVICE)

# 최적화 인스턴스
import torch.optim as optim
OPTIMIZER = optim.Adam(model.parameters())


In [56]:
### ===> 학습 진행 함수
def training():
    # 학습 모드 => 정규화, 경사하강법, 드랍아웃 등의 기능 활성화
    model.train()
    
    # 배치 크기만큼 학습 진행 및 저장
    train_loss = []
    for cnt, (feature, target) in enumerate(trainDL):
        # print(cnt, feature, target)
        # 배치 크기만큼의 학습 데이터 준비      
        feature, target = feature.to(DEVICE), target.to(DEVICE)
        
        target = target.squeeze()
        
        # 학습
        pre_target = model(feature)
        # print(f"pre_target => {pre_target.shape}, {pre_target.ndim}D")
        # print(f"target => {target.shape}, {target.ndim}D")
        
        
        # 손실 계산
        loss = LOSS_FN(pre_target, target)
        train_loss.append(loss)
        
        # W, b 업데이트
        OPTIMIZER.zero_grad()
        loss.backward()
        OPTIMIZER.step()
        
        # 배치 단위 학습 진행 메시지 출력
        # print(f"[Train {cnt} batch LOSS] ===> {loss}")
        
        # 정확도 Accuracy
        acc = metrics.accuracy(pre_target, target, task = 'multiclass', num_classes = OUT)
        # print(f" ACC : {acc}")
        
    # 에포크 단위 학습 진행 메시지 출력
    print(f"[Train loss] ===> {loss}")
    # print(f"ACC : {acc}")
    return train_loss

In [57]:
### ===> 검증 및 평가 진행 함수
# 매개변수 dataLoader : 검증 또는 테스트 데이터셋에 대한 Loader
def testing(dataLoader):
    # 추론모드 => 정규화, 경사하강법, 드랍아웃 등의 기능 비활성화
    model.eval()
    
    with torch.no_grad():
        # 배치 크기 만큼 학습 진행 및 저장
        val_loss = []
        for cnt, (feature, target) in enumerate(dataLoader):
            # 배치 크기 만큼의 학습 데이터 준비
            feature, target = feature.to(DEVICE), target.to(DEVICE)
            target = target.squeeze()
            
            # 학습
            pre_target = model(feature)
            
            # 손실 계산
            # loss = LOSS_FN(pre_target, target) # 필요없음! 정답을 모르기때문에 정답을 계산할 수 없음.
            # val_loss.append(loss)
           
            
        # 에포크 단위 학습 진행 메시지 출력
        # acc = metrics.accuracy(pre_target, target, task = 'multiclass', num_classes = OUT)
        # print(f"[Valid loss] ===> {loss}")
        # print(f"[Valid ACC] : {acc}")
        
        return val_loss

In [58]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

# optimizer 초기화 및 모델과 연결
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# ReduceROnPlateau 스케줄러 설정
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=10, verbose=True)


# 학습 중간에 손실 값을 사용하여 스케줄러 업데이트
for eps in range(EPOCHS):
    # ======================= 학습 =======================
    train_loss = training()
    
    # ======================= 검증 =======================
    valid_loss = testing(validDL)
    
    print(f"[{eps}/{EPOCHS}] TRAIN {sum(train_loss)/len(train_loss)} VALID {sum(valid_loss)/len(valid_loss)}\n")
    
    print(valid_loss)
    # ======================= 학습 스케줄러 =======================
    # 조기 종료 기능 ==> 조건 : val)loss가 지정된 횟수 (예 : 5) 이상 개선이 안되면 학습 종료
    
    # 학습률 조절 : ReduceLROnPlateau 스케줄러에 현재 검증 손실값 전달
    scheduler.step(valid_loss[1])
    
    # 조기 종료 체크
    # 여기서 num_bad_epohcs는 연속적으로 손실이 개선되지 않은 epoch의 수
    if scheduler.num_bad_epochs > scheduler.patience:
        print(f"Early stopping at epoch {eps}")
        break
    

c:\ProgramData\anaconda3\envs\Torch_py38\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float